In [2]:
!pip install transformers datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from transformers import DistilBertTokenizerFast, DistilBertForTokenClassification, Trainer, TrainingArguments
from transformers import XLMRobertaTokenizerFast, XLMRobertaForTokenClassification
from transformers import BertTokenizerFast, BertForTokenClassification
from datasets import load_dataset

# Function to load the dataset
def load_data():
    return load_dataset("conll2003", data_files="/content/drive/MyDrive/EthioMart-/scripts/labeled_data.conll")

# Function to tokenize and align labels
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        # Ensure the labels match the length of the tokenized inputs
        if len(label_ids) < len(tokenized_inputs["input_ids"][i]):
            label_ids += [-100] * (len(tokenized_inputs["input_ids"][i]) - len(label_ids))
        elif len(label_ids) > len(tokenized_inputs["input_ids"][i]):
            label_ids = label_ids[:len(tokenized_inputs["input_ids"][i])]
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Function to train and evaluate a model
def train_and_evaluate_model(model_name, tokenizer, model, train_dataset, eval_dataset):
    # Tokenize the datasets
    tokenized_train_dataset = train_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer), batched=True)
    tokenized_eval_dataset = eval_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer), batched=True)

    # Set up training arguments
    training_args = TrainingArguments(
        output_dir=f"./results/{model_name}",
        evaluation_strategy="epoch",  # Evaluation strategy
        save_strategy="epoch",  # Save strategy set to match evaluation strategy
        learning_rate=2e-5,
        per_device_train_batch_size=4,  # Adjusted for faster training
        per_device_eval_batch_size=4,  # Adjusted for faster evaluation
        num_train_epochs=3,  # Increased to 3 epochs
        weight_decay=0.01,
        logging_dir=f'./logs/{model_name}',
        save_total_limit=2,
        fp16=True,
        dataloader_num_workers=0,  # Minimize the number of workers to reduce overhead
        disable_tqdm=False,
        load_best_model_at_end=True,
        report_to="none",  # Disable reporting to avoid overhead
        logging_steps=10  # Log training loss every 10 steps
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    evaluation_results = trainer.evaluate()
    print(f"{model_name} Evaluation results: {evaluation_results}")
    return evaluation_results

# Load your dataset
dataset = load_data()

# Get label list from dataset
label_list = dataset['train'].features['ner_tags'].feature.names
num_labels = len(label_list)
print(f"Number of labels: {num_labels}, Labels: {label_list}")

# Prepare datasets
train_dataset = dataset['train'].shuffle(seed=42).select(range(500))  # Use a smaller subset of 500 samples for quicker training
eval_dataset = dataset['test'].shuffle(seed=42).select(range(100))  # Use a smaller subset of 100 samples for quicker evaluation

# Fine-tune and evaluate DistilBERT
distilbert_tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-multilingual-cased")
distilbert_model = DistilBertForTokenClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=num_labels)
distilbert_results = train_and_evaluate_model("DistilBERT", distilbert_tokenizer, distilbert_model, train_dataset, eval_dataset)

# Fine-tune and evaluate XLM-Roberta
xlm_roberta_tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
xlm_roberta_model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=num_labels)
xlm_roberta_results = train_and_evaluate_model("XLM-Roberta", xlm_roberta_tokenizer, xlm_roberta_model, train_dataset, eval_dataset)

# Fine-tune and evaluate mBERT
mbart_tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")
mbart_model = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=num_labels)
mbart_results = train_and_evaluate_model("mBERT", mbart_tokenizer, mbart_model, train_dataset, eval_dataset)

# Compare the results
results_comparison = {
    "DistilBERT": distilbert_results['eval_loss'],
    "XLM-Roberta": xlm_roberta_results['eval_loss'],
    "mBERT": mbart_results['eval_loss']
}

# Find the best-performing model
best_model = min(results_comparison, key=results_comparison.get)
print(f"Best performing model is: {best_model} with evaluation loss: {results_comparison[best_model]}")


Number of labels: 9, Labels: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,0.221000,0.238195
2,0.128200,0.138861
3,0.042700,0.122045


...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,0.150500,0.159359
2,0.087600,0.105759
3,0.019800,0.107542


mBERT Evaluation results: {'eval_loss': 0.10575883835554123, 'eval_runtime': 0.4331, 'eval_samples_per_second': 230.887, 'eval_steps_per_second': 57.722, 'epoch': 3.0}
Best performing model is: mBERT with evaluation loss: 0.10575883835554123
